# Historical meteo data importing and cleaning

In further analysis, it would be worth comparing the data from the DIY sensor with historical data from an official weather station that is as close as possible to the location of the sensor.

### Downloading historical data from 1991 to 2021.

All historical data from the official weather stations are available on the IMGW (National Polish Meteo Institute) website in the form of .csv files, which are in the .zip archive.



In [2]:
from bs4 import BeautifulSoup
import time
import requests

In [3]:
# URL path to the online directory where all historical weather data are stored
url_core = 'https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/'

# Variable storing a list with particular directory' names
years = list(range(2001,2022)) + ['1996_2000','1991_1995']

ext = 'zip'     # extension of files with data
path = 'data/'      # name of target directory for downloaded data files

# Simple function for file downloading
def download_file(url):
    r = requests.get(url)
    with open((path + filename), 'wb') as f:
        f.write(r.content)
    
    print(filename + " " + str(r.status_code))
    time.sleep(0.5)     # time interval

# Loop over a list of years to download all the data (it may take a while)
for url in years:
    response = requests.get(url_core + str(url))
    
    soup = BeautifulSoup(response.content, 'html.parser')
    time.sleep(0.5)
    
    for name in soup.select(f"a[href*={ext}]"):
        filename = name.get_text().strip()
        download_url = url_core + str(url) + "/" + filename
        
        download_file(download_url)

2001_01_k.zip 200
2001_02_k.zip 200
2001_03_k.zip 200
2001_04_k.zip 200
2001_05_k.zip 200
2001_06_k.zip 200
2001_07_k.zip 200
2001_08_k.zip 200
2001_09_k.zip 200
2001_10_k.zip 200
2001_11_k.zip 200
2001_12_k.zip 200
2002_01_k.zip 200
2002_02_k.zip 200
2002_03_k.zip 200
2002_04_k.zip 200
2002_05_k.zip 200
2002_06_k.zip 200
2002_07_k.zip 200
2002_08_k.zip 200
2002_09_k.zip 200
2002_10_k.zip 200
2002_11_k.zip 200
2002_12_k.zip 200
2003_01_k.zip 200
2003_02_k.zip 200
2003_03_k.zip 200
2003_04_k.zip 200
2003_05_k.zip 200
2003_06_k.zip 200
2003_07_k.zip 200
2003_08_k.zip 200
2003_09_k.zip 200
2003_10_k.zip 200
2003_11_k.zip 200
2003_12_k.zip 200
2004_01_k.zip 200
2004_02_k.zip 200
2004_03_k.zip 200
2004_04_k.zip 200
2004_05_k.zip 200
2004_06_k.zip 200
2004_07_k.zip 200
2004_08_k.zip 200
2004_09_k.zip 200
2004_10_k.zip 200
2004_11_k.zip 200
2004_12_k.zip 200
2005_01_k.zip 200
2005_02_k.zip 200
2005_03_k.zip 200
2005_04_k.zip 200
2005_05_k.zip 200
2005_06_k.zip 200
2005_07_k.zip 200
2005_08_k.

Unzipping all files and removing remaining .zip files

In [3]:
import zipfile, os, glob

In [5]:
dir_name = 'data'
extension = ".zip"

file_list = os.listdir(dir_name)

os.chdir(dir_name) 

for item in file_list: 
    if item.endswith(extension): 
        file_name = os.path.abspath(item) 
        zip_ref = zipfile.ZipFile(file_name) 
        zip_ref.extractall(dir_name) 
        zip_ref.close() 
        os.remove(file_name) 

Each .zip file has two .csv files inside. We are only interested in files with 'k_d_' prefix so other files with 'k_d_t_' prefix may be removed.

In [6]:
fileList = glob.glob('data/k_d_t_*.csv')

for filePath in fileList:
    os.remove(filePath)

### Creating one dataframe with data from all .csv files


Quick preview of the random single .csv file. We need to upload all of the data into one dataframe with correct columns names which can be found in the dataset documentation. 

In [1]:
import pandas as pd

In [9]:
pd.read_csv('data/k_d_01_2001.csv', encoding='latin2').sample(5)

,249180010,PSZCZYNA,2001,01,01.1,-1.3,Unnamed: 6,-9.6,Unnamed: 8,-5.7,Unnamed: 10,-11.0,Unnamed: 12,.0,9,Unnamed: 15,13,Unnamed: 17
814,249200410,PIWNICZNA,2001,1,10,2.4,NaN,-3.8,NaN,-1.5,NaN,0.0,8.0,0.5,NaN,S,8,NaN
4014,253160090,WIERZCHOWO,2001,1,17,-1.0,NaN,-3.2,NaN,-2.4,NaN,0.0,8.0,0.0,9.0,NaN,0,9.0
4651,254180090,GDAŃSK-RĘBIECHOWO,2001,1,3,1.7,NaN,0.0,NaN,0.9,NaN,-0.3,NaN,0.0,NaN,W,15,NaN
2987,251170150,GRABOWNICA,2001,1,13,0.3,NaN,-5.0,NaN,-2.7,NaN,0.0,8.0,0.0,9.0,NaN,0,9.0
4027,253160090,WIERZCHOWO,2001,1,30,2.2,NaN,-1.3,NaN,0.0,NaN,0.0,8.0,2.9,NaN,S,0,9.0


The next task is to combine all .csv files into one dataframe. We are only interested in the data for the PUŁAWY weather station, because it is the closest to the place (~22 km) where the NodeMCU sensor is installed.

In [4]:
# Correct columns names found in the dataset documentation
colnames = ['station_id','station_name','year','month','day','t_max','t_max_status','t_min','t_min_status','t_mean','t_mean_status','t_ground','t_ground_status','percip','percip_status','percip_type','snow','snow_status']

df = pd.DataFrame()

for i in glob.glob('data/data/k_d*.csv'):
    temp = pd.read_csv(i, encoding='latin2', names=colnames)
    temp = temp.loc[temp['station_id'] == 251210120]    # Selecting data rows for PUŁAWY station only (by station id)
    df = pd.concat([df, temp])
    temp = pd.DataFrame()

In [5]:
df.sample(10)

,station_id,station_name,year,month,day,t_max,t_max_status,t_min,t_min_status,t_mean,t_mean_status,t_ground,t_ground_status,percip,percip_status,percip_type,snow,snow_status
3202,251210120,PUŁAWY,2014,12,10,-1.5,NaN,-5.9,NaN,-4.4,NaN,-7.6,NaN,0.0,9.0,NaN,0,9.0
3244,251210120,PUŁAWY,2010,3,21,17.8,NaN,6.3,NaN,11.6,NaN,7.8,NaN,3.1,NaN,W,0,9.0
3242,251210120,PUŁAWY,2010,12,19,-5.0,NaN,-9.4,NaN,-6.9,NaN,-10.8,NaN,0.0,NaN,W,23,NaN
1418,251210120,PUŁAWY,2019,5,24,19.1,NaN,12.2,NaN,15.2,NaN,11.5,NaN,0.0,9.0,NaN,0,9.0
43233,251210120,PUŁAWY,1996,1,18,-3.7,NaN,-8.2,NaN,-5.9,NaN,-12.6,NaN,0.0,NaN,W,2,NaN
1495,251210120,PUŁAWY,2018,1,8,0.6,NaN,-4.4,NaN,-2.8,NaN,-7.0,NaN,0.0,9.0,NaN,0,9.0
3236,251210120,PUŁAWY,2014,1,13,3.5,NaN,-3.2,NaN,-0.8,NaN,-5.9,NaN,0.0,9.0,NaN,0,9.0
3226,251210120,PUŁAWY,2007,4,17,20.9,NaN,4.3,NaN,12.5,NaN,0.8,NaN,1.0,NaN,W,0,9.0
3229,251210120,PUŁAWY,2010,10,6,12.6,NaN,3.4,NaN,6.8,NaN,-0.3,NaN,0.0,9.0,NaN,0,9.0
40305,251210120,PUŁAWY,1999,3,6,11.4,NaN,5.4,NaN,7.1,NaN,0.6,NaN,0.0,9.0,NaN,0,9.0


In [6]:
# Dropping unnecessary columns and leaving only essential

df = df[['year','month','day','t_max','t_min','t_mean']]

In [8]:
df

,year,month,day,t_max,t_min,t_mean
3224,2001,1,1,6.8,-5.2,-1.4
3225,2001,1,2,2.2,-6.3,-1.2
3226,2001,1,3,2.1,-1.0,0.5
3227,2001,1,4,2.7,0.5,1.5
3228,2001,1,5,2.4,-1.0,0.6
...,...,...,...,...,...,...
39241,2000,12,27,-0.3,-2.0,-1.2
39242,2000,12,28,2.8,-0.6,1.3
39243,2000,12,29,3.3,-2.9,-0.4
39244,2000,12,30,1.8,-3.0,0.1


In [9]:
# Converting year, month, day column into a datetime format

df['date'] = pd.to_datetime(df[['year','month','day']].astype(str).apply('-'.join, 1), format='%Y-%m-%d')
df

,year,month,day,t_max,t_min,t_mean,date
3224,2001,1,1,6.8,-5.2,-1.4,2001-01-01
3225,2001,1,2,2.2,-6.3,-1.2,2001-01-02
3226,2001,1,3,2.1,-1.0,0.5,2001-01-03
3227,2001,1,4,2.7,0.5,1.5,2001-01-04
3228,2001,1,5,2.4,-1.0,0.6,2001-01-05
...,...,...,...,...,...,...,...
39241,2000,12,27,-0.3,-2.0,-1.2,2000-12-27
39242,2000,12,28,2.8,-0.6,1.3,2000-12-28
39243,2000,12,29,3.3,-2.9,-0.4,2000-12-29
39244,2000,12,30,1.8,-3.0,0.1,2000-12-30


In [10]:
# Dropping unnecessary columns and sorting rows by date

df = df[['date','t_max','t_min','t_mean']].sort_values(by='date', ignore_index=True)
df

,date,t_max,t_min,t_mean
0,1991-01-01,3.0,-1.5,1.2
1,1991-01-02,4.5,1.0,2.6
2,1991-01-03,10.0,1.5,5.0
3,1991-01-04,6.8,3.0,4.7
4,1991-01-05,6.2,1.0,4.1
...,...,...,...,...
11287,2021-12-27,-4.4,-14.2,-9.6
11288,2021-12-28,-3.2,-8.3,-5.5
11289,2021-12-29,-1.1,-4.9,-2.8
11290,2021-12-30,3.0,-2.2,1.1


Saving the dataframe to a .csv file.

In [65]:
df.to_csv('data/pulawy_data.csv', columns=['date','t_max','t_min','t_mean'], header=['date','t_max','t_min','t_mean'], index=False)

### Calculating mean temperature values for 30 years for PUŁAWY station

Currently we have temperature data for each day. The next step is to calculate the average values for this period.

In [11]:
df2 = df.groupby([df['date'].dt.month,df['date'].dt.day])[['t_max','t_min','t_mean']].agg('mean')
df2

t_max     t_min    t_mean
date date                              
1    1     1.193548 -3.787097 -1.480645
     2     1.380645 -4.329032 -1.632258
     3     0.541935 -4.377419 -2.048387
     4     0.680645 -4.419355 -1.938710
     5     0.667742 -4.874194 -2.190323
...             ...       ...       ...
12   27    1.941935 -3.864516 -1.129032
     28    1.967742 -3.116129 -0.674194
     29    1.812903 -3.235484 -0.948387
     30    1.148387 -3.725806 -1.351613
     31    1.806452 -4.032258 -1.358065

[366 rows x 3 columns]

Using the groupby method, we get the average values in a dataframe with multiindex. Next, we will create a new column 'date', which will contain the specific date in the datetime format that the average values refer to.

In [34]:
df2['date'] = pd.to_datetime('2021-' + df2.index.get_level_values(0).astype(str) + '-' +
               df2.index.get_level_values(1).astype(str),
               format='%Y-%m-%d',errors='coerce')

In [35]:
df2

t_max     t_min    t_mean       date
date date                                         
1    1     1.193548 -3.787097 -1.480645 2021-01-01
     2     1.380645 -4.329032 -1.632258 2021-01-02
     3     0.541935 -4.377419 -2.048387 2021-01-03
     4     0.680645 -4.419355 -1.938710 2021-01-04
     5     0.667742 -4.874194 -2.190323 2021-01-05
...             ...       ...       ...        ...
12   27    1.941935 -3.864516 -1.129032 2021-12-27
     28    1.967742 -3.116129 -0.674194 2021-12-28
     29    1.812903 -3.235484 -0.948387 2021-12-29
     30    1.148387 -3.725806 -1.351613 2021-12-30
     31    1.806452 -4.032258 -1.358065 2021-12-31

[366 rows x 4 columns]

In [43]:
# Removing multiindex from dataframe by dropping first level and reseting index

df2 = df2.droplevel(0)
df2 = df2.reset_index(drop=True)

In [51]:
# Changing a columns order

df2 = df2[['date','t_max','t_min','t_mean']]
df2

,date,t_max,t_min,t_mean
0,2021-01-01,1.193548,-3.787097,-1.480645
1,2021-01-02,1.380645,-4.329032,-1.632258
2,2021-01-03,0.541935,-4.377419,-2.048387
3,2021-01-04,0.680645,-4.419355,-1.938710
4,2021-01-05,0.667742,-4.874194,-2.190323
...,...,...,...,...
361,2021-12-27,1.941935,-3.864516,-1.129032
362,2021-12-28,1.967742,-3.116129,-0.674194
363,2021-12-29,1.812903,-3.235484,-0.948387
364,2021-12-30,1.148387,-3.725806,-1.351613


In [53]:
# Chcecking the data types in the final dataframe

df2.dtypes

date      datetime64[ns]
t_max            float64
t_min            float64
t_mean           float64
dtype: object

In [54]:
# Saving the dataframe to a .csv file

df2.to_csv('data/imgw_30ymean_data.csv', columns=['date','t_max','t_min','t_mean'], header=['date','t_max','t_min','t_mean'], index=False)
